*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [2]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import string

print(tf.__version__)

2.20.0-dev20250423


In [9]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-04-24 17:57:04--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.08s   

2025-04-24 17:57:04 (4.14 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-04-24 17:57:04--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [4]:
# data Loading and preprocessing

train_data = pd.read_csv(train_file_path, sep='\t', header=None)
test_data = pd.read_csv(test_file_path, sep='\t', header=None)
train_data.columns = ['label', 'message']
test_data.columns = ['label', 'message']

# lowercase
def preprocess_text(text):
    text = text.lower()
    return text

train_data['message'] = train_data['message'].apply(preprocess_text)
test_data['message'] = test_data['message'].apply(preprocess_text)

# convert labels to numerical values
train_data['label_num'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label_num'] = test_data['label'].map({'ham': 0, 'spam': 1})

In [5]:
# text vectorization
vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000, output_sequence_length=100)
vectorizer.adapt(train_data['message'])

X_train = vectorizer(train_data['message'])
X_test = vectorizer(test_data['message'])
y_train = train_data['label_num']
y_test = test_data['label_num']

# Compute class weights to handle imbalance
num_ham = (y_train == 0).sum()
num_spam = (y_train == 1).sum()
total = len(y_train)
weight_for_0 = (1 / num_ham) * (total / 2.0)
weight_for_1 = (1 / num_spam) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}

In [6]:
# build model
model = keras.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=32),
    keras.layers.Bidirectional(keras.layers.LSTM(32)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test),
          class_weight=class_weight, verbose=0)

In [7]:
# function to predict messages based on model
def predict_message(pred_text):
    pred_text = preprocess_text(pred_text)
    pred_vector = vectorizer([pred_text])
    prediction = model.predict(pred_vector, verbose=0)[0][0]
    label = 'spam' if prediction > 0.5 else 'ham'
    return [prediction, label]

In [8]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
